# Notebook für SDM Analyse

## Load packages

In [ ]:
# import standard packages
from datetime import date, datetime
import os
from pathlib import Path

import ee
import fiona
import geemap #.foliumap as geemap
import geemap.foliumap as folium
import geemap.colormaps as cm
import geopandas as gpd
import matplotlib.pyplot as plt
import matplotlib as mpl

import numpy as np

import requests

from scipy.special import gamma, betainc
import seaborn as sns
import shapely
from statsmodels.stats.outliers_influence import variance_inflation_factor
from sklearn.metrics import confusion_matrix, ConfusionMatrixDisplay, classification_report
import pandas as pd
from pygbif import species as species
from pygbif import occurrences as occ

import rioxarray as rxr
import xarray as xr
import leafmap



from toolbox.utils import calculate_nari, calculate_ncri, calculate_ndvi, gee_calculate_scrub_index, get_species_data, remove_duplicates, mask_s2_clouds, load_background_data, get_layer_information, get_aoi_from_nuts, get_layer_visualization_params, plot_correlation_heatmap, compute_sdm, plot_hier_clustering

2026-01-04 21:03:23.797 WARNING streamlit.runtime.caching.cache_data_api: No runtime found, using MemoryCacheStorageManager
2026-01-04 21:03:23.797 WARNING streamlit.runtime.caching.cache_data_api: No runtime found, using MemoryCacheStorageManager
2026-01-04 21:03:23.797 WARNING streamlit.runtime.caching.cache_data_api: No runtime found, using MemoryCacheStorageManager
